In [1]:
# imports
import os
import gc
import random
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import KFold

import lightgbm as lgb

tqdm.pandas()
%matplotlib inline


import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 2021
seed_everything(SEED)

In [2]:
# feature utils
def calculate_wap(df, rank="1"):
    """
    Weighted Average Pricing for a stock at a given time ID is given by:
    (bid_price1 * ask_size1 + bid_size1 * ask_price1)/(bid_size1 + ask_size1)

    It can further be extended to:

        sum(bid_price_i * ask_size_i + bid_size_i * ask_price_i)/sum(bid_size_i + ask_size_i)

    :param rank: which wap to calculate
    :param df: parquet table containing order book
    :return:
    """
    return (df[f"bid_price{rank}"] * df[f"ask_size{rank}"] + df[f"bid_size{rank}"] * df[f"ask_price{rank}"]) / (
                df[f"bid_size{rank}"] + df[f"ask_size{rank}"])


def calculate_inter_wap(df, rank="1"):
    return (df[f"bid_price{rank}"] * df[f"bid_size{rank}"] + df[f"ask_size{rank}"] * df[f"ask_price{rank}"]) / (
            df[f"bid_size{rank}"] + df[f"ask_size{rank}"])
    pass


def calculate_log_return(series):
    return np.log(series).diff()


def calculate_rv(series):
    return np.sqrt(np.sum(np.square(series)))


def count_unique(series):
    return len(np.unique(series))


def get_stats_window(df, seconds_in_bucket, features_dict, add_suffix=False):
    df_feature = df[df["seconds_in_bucket"] >= seconds_in_bucket].groupby(["time_id"]).agg(features_dict).reset_index()
    df_feature.columns = ["_".join(col) for col in df_feature.columns]

    if add_suffix:
        df_feature = df_feature.add_suffix("_" + str(seconds_in_bucket))

    return df_feature
    pass


def window_stats(df, feature_dict, buck_feats, second_windows, additional_dfs=None):
    df_merged = get_stats_window(df, seconds_in_bucket=0, features_dict=feature_dict)

    if additional_dfs is not None:
        df_merged = df_merged.merge(additional_dfs, how='left', left_on='time_id_', right_on='time_id')

    temp_dfs = []
    for window in second_windows:
        temp_dfs.append(
            (window,
             get_stats_window(df, seconds_in_bucket=window, features_dict=buck_feats, add_suffix=True)
             )
        )

    for window, temp_df in temp_dfs:
        df_merged = df_merged.merge(temp_df, how="left", left_on="time_id_", right_on=f"time_id__{window}")
        df_merged.drop(columns=[f"time_id__{window}"], inplace=True)

    return df_merged
    pass


def tendency(price, vol):
    diff = np.diff(price)
    val = (diff / price[1:]) * 100
    power = np.sum(val * vol[1:])
    return power
    pass


def get_stock_clusters(df, n_clusters=7):
    pivoted_data = df.pivot(index="time_id", columns=["stock_id"], values="target")
    corr_pivoted = pivoted_data.corr()

    clusters = KMeans(n_clusters, random_state=cfg.random_state).fit(corr_pivoted.values)

    groups = []
    for i in range(n_clusters):
        groups.append([x-1] for x in (corr_pivoted.index+1)*(clusters.labels_ == i) if x > 0)
    return groups
    pass


def create_cluster_aggregations(df, groups):
    feats = []

    for i, idx in enumerate(groups):
        chunk_df = df.loc[df['stock_id'].isin(idx)]
        chunk_df = chunk_df.groupby(['time_id']).agg(np.nanmean)
        chunk_df.loc[:, 'stock_id'] = str(i) + 'c1'
        feats.append(chunk_df)

    feats = pd.concat(feats).reset_index()
    if "target" in feats.columns:
        feats.drop(columns=['target'], inplace=True)

    feats = feats.pivot(index='time_id', columns='stock_id')
    feats.columns = ["_".join(x) for x in feats.columns.ravel()]
    feats.reset_index(inplace=True)

    return pd.merge(df, feats, how="left", on="time_id")
    pass

In [3]:
# config
class cfg:
    
    paths = {
        # train path
        "train_csv": "../input/optiver-realized-volatility-prediction/train.csv",
        "train_book": "../input/optiver-realized-volatility-prediction/book_train.parquet",
        "train_trade": "../input/optiver-realized-volatility-prediction/trade_train.parquet",

        # test path
        "test_csv": "../input/optiver-realized-volatility-prediction/test.csv",
        "test_book": "../input/optiver-realized-volatility-prediction/book_test.parquet",
        "test_trade": "../input/optiver-realized-volatility-prediction/trade_test.parquet",

        # model paths
        "xgb_baseline": "./xgbBaseline/",
        "lgb_baseline": "./lgbBaseline"  # for inference
    }

    feature_dict_book = {
        'seconds_in_bucket': [count_unique],
        "wap1": [np.sum, np.mean, np.max],
        "wap2": [np.sum, np.mean, np.max],
        "iwap1": [np.sum, np.mean, np.max],
        "iwap2": [np.sum, np.mean, np.max],
        "log_return1": [np.sum, calculate_rv, np.mean, np.max],
        "log_return2": [np.sum, calculate_rv, np.mean, np.max],
        "inter_log_return1": [np.sum, calculate_rv, np.mean, np.max],
        "inter_log_return2": [np.sum, calculate_rv, np.mean, np.max],
        "wap_balance": [np.sum, np.mean, np.max],
        "volume_imbalance": [np.sum, np.mean, np.max],
        "total_volume": [np.sum, np.mean, np.max],
        "price_spread1": [np.sum, np.mean, np.max],
        "price_spread2": [np.sum, np.mean, np.max],
        "bid_spread": [np.sum, np.mean, np.max],
        "ask_spread": [np.sum, np.mean, np.max],
    }
    
    feature_dict_book_time = {
        "log_return1": [calculate_rv],
        "log_return2": [calculate_rv],
        "inter_log_return1": [calculate_rv],
        "inter_log_return2": [calculate_rv],
        
    }

    feature_dict_trade = {
        'log_return': [np.sum, calculate_rv, np.mean, np.max],
        'seconds_in_bucket': [count_unique],
        'size': [np.sum, np.mean, np.max],
        'order_count': [np.sum, np.mean, np.max],
        'amount': [np.sum, np.mean, np.max],
    }

    feature_dict_trade_time = {
        'log_return': [calculate_rv],
        'seconds_in_bucket': [count_unique],
        'size': [np.sum],
        'order_count': [np.sum],
        'amount': [np.mean],
    }
    
    model_params = {
        "xgb_bl": {
            "objective": "reg:squarederror",
            "booster": "gbtree",
            "nthread": -1,
            "eta": 0.3,
            "max_depth": 8,
            "min_child_weight": 1,
            "sampling_method": "uniform",
            # "tree_method": "gpu_hist"
        },
        "lgb_bl": {
            "objective": "rmse",
            "boosting_type": "gbdt",
            "learning_rate": 0.05,
            'max_depth': -1,
            'max_bin':100,
            'min_data_in_leaf':500,
            'learning_rate': 0.05,
            'subsample': 0.72,
            'subsample_freq': 4,
            'feature_fraction': 0.5,
            'lambda_l1': 0.5,
            'lambda_l2': 1.0,
            
            "categorical_column": [0],
            "n_jobs": -1,
            "verbose": -1,
            "seed": SEED,
            "feature_fraction_seed": SEED,
            "bagging_seed": SEED,
            "drop_seed": SEED,
            "data_random_seed": SEED,
        }
    }

    bucket_windows = [100, 200, 300, 400, 500]
    random_state = SEED
    pass

In [4]:
# order book features
def get_book_features(file_path):
    book_df = pd.read_parquet(file_path)

    # calculate wap
    book_df['wap1'] = calculate_wap(book_df, rank="1")
    book_df['wap2'] = calculate_wap(book_df, rank="2")
    book_df['iwap1'] = calculate_inter_wap(book_df, rank="1")
    book_df['iwap2'] = calculate_inter_wap(book_df, rank="2")

    # calculate log return
    book_df["log_return1"] = book_df.groupby(["time_id"])["wap1"].apply(calculate_log_return)
    book_df["log_return2"] = book_df.groupby(["time_id"])["wap2"].apply(calculate_log_return)
    book_df["inter_log_return1"] = book_df.groupby(["time_id"])["iwap1"].apply(calculate_log_return)
    book_df["inter_log_return2"] = book_df.groupby(["time_id"])["iwap2"].apply(calculate_log_return)

    # calculate balance
    book_df["wap_balance"] = abs(book_df["wap1"] - book_df["wap2"])
    book_df["volume_imbalance"] = abs(
        (book_df["ask_size1"] + book_df["ask_size2"]) - (book_df["bid_size1"] + book_df["bid_size2"])
    )
    book_df["total_volume"] = book_df["ask_size1"] + book_df["ask_size2"] + book_df["bid_size1"] + book_df[
        "bid_size2"]

    # calculate spread
    book_df["price_spread1"] = (book_df["ask_price1"] - book_df["bid_price1"]) / (
            (book_df["ask_price1"] + book_df["bid_price1"]) / 2)
    book_df["price_spread2"] = (book_df["ask_price2"] - book_df["bid_price2"]) / (
            (book_df["ask_price2"] + book_df["bid_price2"]) / 2)

    book_df["bid_spread"] = book_df["bid_price1"] - book_df["bid_price2"]
    book_df["ask_spread"] = book_df["ask_price1"] - book_df["ask_price2"]

    book_df_merged = window_stats(book_df, cfg.feature_dict_book, cfg.feature_dict_book_time, cfg.bucket_windows)

    book_df_merged["row_id"] = book_df_merged["time_id_"].apply(lambda x: f"{file_path.split('=')[1]}-{x}")
    book_df_merged.drop(["time_id_"], axis=1, inplace=True)

    return book_df_merged.bfill().ffill()
                                                                
# trade features
def get_trade_price_features(df):
    res = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]
        vol_tendency = tendency(df_id['price'].values, df_id['size'].values)
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max = np.sum(np.diff(df_id['price'].values) > 0)
        df_min = np.sum(np.diff(df_id['price'].values) < 0)
        abs_diff = np.median(np.abs(df_id['price'].values - np.mean(df_id['price'].values)))
        energy = np.mean(df_id['price'].values ** 2)
        iqr_p = np.percentile(df_id['price'].values, 75) - np.percentile(df_id['price'].values, 25)
        abs_diff_v = np.median(np.abs(df_id['size'].values - np.mean(df_id['size'].values)))
        energy_v = np.sum(df_id['size'].values ** 2)
        iqr_p_v = np.percentile(df_id['size'].values, 75) - np.percentile(df_id['size'].values, 25)

        res.append({'time_id': n_time_id,
                    'tendency': vol_tendency,
                    'f_max': f_max,
                    'f_min': f_min,
                    'df_max': df_max,
                    'df_min': df_min,
                    'abs_diff': abs_diff,
                    'energy': energy,
                    'iqr_p': iqr_p,
                    'abs_diff_v': abs_diff_v,
                    'energy_v': energy_v,
                    'iqr_p_v': iqr_p_v})

    return pd.DataFrame(res)
    pass


def tau_features(df, sec, weight):
    tau_feat = 'tau_' + str(sec)
    bucket_col = 'trade_seconds_in_bucket_count_unique_' + str(sec)
    df[tau_feat] = np.sqrt(weight/df[bucket_col])

    size_feat = 'size_' + str(sec)
    order_col = 'trade_order_count_sum_' + str(sec)
    df[size_feat] = np.sqrt(weight/df[order_col])

    return df
    pass


def get_trade_features(file_path, buck_windows=cfg.bucket_windows):
    trade_df = pd.read_parquet(file_path)

    trade_df["log_return"] = trade_df.groupby(["time_id"])["price"].apply(calculate_log_return)
    trade_df["amount"] = trade_df["size"] * trade_df["price"]

    price_features = get_trade_price_features(trade_df)
    trade_df_merged = window_stats(trade_df, cfg.feature_dict_trade, cfg.feature_dict_trade_time, buck_windows, additional_dfs=price_features)

    trade_df_merged = trade_df_merged.add_prefix("trade_")

    trade_df_merged["row_id"] = trade_df_merged["trade_time_id_"].apply(lambda x: f"{file_path.split('=')[1]}-{x}")
    trade_df_merged.drop(["trade_time_id_"], axis=1, inplace=True)

    for sec in buck_windows:
        trade_df_merged = tau_features(trade_df_merged, sec, weight=sec/600)
    return trade_df_merged.bfill().ffill()                                              

In [5]:
# create dataset
class GetData:
    def __init__(self, df, book_path, trade_path, is_train=True):
        self.df = df.copy(deep=True)
        self.order_book_path = book_path
        self.trade_path = trade_path
        self.is_train = is_train

        self._get_rowid()

    def _get_rowid(self):
        self.df["row_id"] = self.df["stock_id"].astype(str) + "-" + self.df["time_id"].astype(str)

    def get_time_stock(self, buck_windows=cfg.bucket_windows):
        vol_cols = []
        feat_set = ['log_return1_calculate_rv', 'log_return2_calculate_rv', 'trade_log_return_calculate_rv']
        for feat in feat_set:
            for sec in buck_windows:
                vol_cols.append(feat + f'_{sec}')
        vol_cols += feat_set

        df_stock_id = self.df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
        df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
        df_stock_id = df_stock_id.add_suffix('_' + 'stock')

        df_time_id = self.df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
        df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
        df_time_id = df_time_id.add_suffix('_' + 'time')

        # Merge with original dataframe
        self.df = self.df.merge(df_stock_id, how='left', left_on=['stock_id'], right_on=['stock_id__stock'])
        self.df = self.df.merge(df_time_id, how='left', left_on=['time_id'], right_on=['time_id__time'])
        self.df.drop(['stock_id__stock', 'time_id__time'], axis=1, inplace=True)
        return self.df

    def process_features(self, list_stock_ids):
        def parallel_helper(stock_id):
            book_sample_path = os.path.join(self.order_book_path, f"stock_id={stock_id}")
            trade_sample_path = os.path.join(self.trade_path, f"stock_id={stock_id}")

            return pd.merge(get_book_features(book_sample_path), get_trade_features(trade_sample_path),
                            on="row_id",
                            how="left")

        df = Parallel(n_jobs=4, verbose=1)(delayed(parallel_helper)(stock_id) for stock_id in list_stock_ids)
        df = pd.concat(df, ignore_index=True)

        return df

    def _get_features(self):
        features_df = self.process_features(self.df["stock_id"].unique())
        self.df = self.df.merge(features_df, on=["row_id"], how="left")

        return self.get_time_stock()
        pass

    def get_all_features(self, stock_groups):
        return create_cluster_aggregations(self._get_features(), stock_groups).bfill().ffill()
        pass

In [6]:
# metric utils
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, model, is_xgb=True):
    y_true = model.get_label()

    if is_xgb:
        return "RMSPE", rmspe(y_true, y_pred)

    return "RMSPE", rmspe(y_true, y_pred), False

In [7]:
# model utils
def feval_wrapper(y_pred, model):
    return feval_rmspe(y_pred, model, is_xgb=False)

class TrainFer:
    def __init__(self, params_dict, n_splits, model_path, random_state):
        self.params = params_dict
        self.n_splits = n_splits
        self.random_state = random_state
        self.model_path = model_path
        if not os.path.isdir(model_path):
            os.makedirs(model_path)

    def train(self, X, y, scaler=None):
        
        if scaler is not None:
            X.iloc[:, 1:] = scaler.fit_transform(X.iloc[:, 1:])
            pickle.dump(scaler, open(os.path.join(self.model_path, "scaler.pkl"), "wb"))
            
        oof_predictions = np.zeros(X.shape[0])
        kfold = KFold(n_splits=self.n_splits, random_state=0, shuffle=True)
        oof_scores = []

        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f"\nFold - {fold}\n")

            x_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            x_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            dtrain = lgb.Dataset(x_train, y_train, weight=1/np.square(y_train))
            dval = lgb.Dataset(x_val, y_val, weight=1/np.square(y_val))

            model = lgb.train(params=self.params,
                              num_boost_round=10000,
                              train_set=dtrain,
                              valid_sets=[dtrain, dval],
                              verbose_eval=250,
                              early_stopping_rounds=200,
                              feval=feval_wrapper)

            fold_preds = model.predict(x_val)
            oof_score = rmspe(y_val, fold_preds)
            print(f"\nRMSPE of fold {fold}: {oof_score}")
            pickle.dump(model, open(os.path.join(self.model_path, f"lgb_bl_{fold}_{oof_score}.pkl"), "wb"))
            
            oof_scores.append(oof_score)
            oof_predictions[val_idx] = fold_preds
        
        print(f"\nOOF Scores: {oof_scores}\n")
        rmspe_score = rmspe(y, oof_predictions)
        print(f"OOF RMSPE: {rmspe_score}")
        
    def infer(self, x_test):        
        test_predictions = np.zeros(x_test.shape[0])

        for mpth in os.listdir(self.model_path):
            model = pickle.load(open(os.path.join(self.model_path, mpth), "rb"))
            test_predictions += model.predict(x_test)/5

        return test_predictions

In [8]:
if __name__=="__main__":
    _ = gc.collect()
    is_train = True
    
    train = pd.read_csv(cfg.paths["train_csv"])
    groups = get_stock_clusters(train)
    
    model = TrainFer(cfg.model_params["lgb_bl"], n_splits=5, model_path=cfg.paths["lgb_baseline"], random_state=cfg.random_state)
    
    if is_train:
        train_data = GetData(train, cfg.paths["train_book"], cfg.paths["train_trade"])
        train_df = train_data.get_all_features(groups)
        pickle.dump(train_df, open("./processed_train.pkl", "wb"))
#         train_df = pickle.load(open("./processed_train.pkl", "rb"))
        model.train(train_df.drop(columns=["row_id", "target", "time_id"]), train_df["target"], scaler=StandardScaler())
    else:
        test = pd.read_csv(cfg.paths["test_csv"])
        test_data = GetData(test, cfg.paths["test_book"], cfg.paths["test_trade"])
        test_df = test_data.get_all_features(groups)
        
        preds = model.infer(test_df.drop(columns=["row_id", "time_id"])) 
        test["target"] = preds
        test[["row_id", "target"]].to_csv("./submission.csv", index=False)
        print(test.head())

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 16.0min
[Parallel(n_jobs=4)]: Done 112 out of 112 | elapsed: 41.2min finished



Fold - 0

Training until validation scores don't improve for 200 rounds
[250]	training's rmse: 0.00044239	training's RMSPE: 0.204678	valid_1's rmse: 0.000454718	valid_1's RMSPE: 0.210718
[500]	training's rmse: 0.000418936	training's RMSPE: 0.193827	valid_1's rmse: 0.000438522	valid_1's RMSPE: 0.203213
[750]	training's rmse: 0.000404279	training's RMSPE: 0.187046	valid_1's rmse: 0.000430068	valid_1's RMSPE: 0.199295
[1000]	training's rmse: 0.000393672	training's RMSPE: 0.182138	valid_1's rmse: 0.000425258	valid_1's RMSPE: 0.197067
[1250]	training's rmse: 0.000384897	training's RMSPE: 0.178078	valid_1's rmse: 0.000421909	valid_1's RMSPE: 0.195514
[1500]	training's rmse: 0.000377444	training's RMSPE: 0.17463	valid_1's rmse: 0.000419155	valid_1's RMSPE: 0.194238
[1750]	training's rmse: 0.000371155	training's RMSPE: 0.17172	valid_1's rmse: 0.000417527	valid_1's RMSPE: 0.193484
[2000]	training's rmse: 0.000365725	training's RMSPE: 0.169208	valid_1's rmse: 0.000416637	valid_1's RMSPE: 0.1930

EOF